In [1]:
import pandas
import numpy
import statsmodels.api as sm
%run overdispersion-test.ipynb
%run Table_Summary-poisson.ipynb
from scipy.stats import chi2, f
import warnings
from statsmodels.tools.sm_exceptions import ValueWarning
warnings.simplefilter('ignore',ValueWarning)

## Hybrid sample

In [2]:
filepath = 'D:/BG/Data/Processing/4_data.txt'
data = pandas.read_csv(filepath, sep='\t')
print(data.shape[0],'job postings in full sample')
hybrid = data[data.plant_5==0]
print(hybrid.shape[0],'job postings in hybrid sample')

3091485 job postings in full sample
212822 job postings in hybrid sample


## Preparing data

In [3]:
df = hybrid.copy()
df.reset_index(inplace=True,drop=True)
measures = ['Engineering Skills','Operations Skills','Support Skills','General Skills','Job Complexity']
df['Engineering Skills'] = df['research']+df['design']+df['materials']+df['development']
df['Operations Skills'] = df['tools']+df['inventory']+df['production']
df['Support Skills'] = df['business']+df['finance']+df['management']+df['analysis']+df['customer']+df['office']+df['software']
df['General Skills'] = df['cognitive']+df['social']
df['Job Complexity'] = df['complexity']
Z = df['plant'].tolist()
df['Year'] = pandas.to_datetime(df.JobDate).dt.year
df.loc[df.Year<2017,'Period'] = 0
df.loc[df.Year>2016,'Period'] = 1
X = df[['TECH','occupation','plant','Period']]
X = pandas.get_dummies(X,columns=['TECH','occupation','plant'])
X = X.drop(['TECH_TM','occupation_Operator','plant_United Technolo East Hartford'],axis=1)
X = X.rename(columns={'occupation_Manager':'Manager','occupation_Engineer':'Engineer','occupation_Technician':'Technician','TECH_AM':'TECH'})
occ = ['Manager','Engineer','Technician']
op = []
ao = []
apo = []
for o in occ:
  X['TECH*'+o] = X['TECH']*X[o]
  X[o+'*Period'] = X[o]*X['Period']
  X['TECH*Period*'+o] = X['TECH']*X['Period']*X[o]
  op.append(o+'*Period')
  ao.append('TECH*'+o)
  apo.append('TECH*Period*'+o)
X['TECH*Period'] = X['TECH'] * X['Period']
X['Intercept'] = 1
listC = ['TECH'] + occ + ['Period'] + ao + ['TECH*Period'] + op + apo + ['Intercept']
listB = [var for var in listC if var not in apo]
listA = [var for var in listB if var not in op + ao + ['TECH*Period']]
all_variables = listC + (X.columns.drop(listC).tolist())
XC = X[all_variables]
XB = XC.drop(apo,axis=1)
XA = XB.drop(op + ao + ['TECH*Period'],axis=1)
print(XA.shape)
print(XB.shape)
print(XC.shape)
print(XC.columns[0:25])

(212822, 324)
(212822, 331)
(212822, 334)
Index(['TECH', 'Manager', 'Engineer', 'Technician', 'Period', 'TECH*Manager',
       'TECH*Engineer', 'TECH*Technician', 'TECH*Period', 'Manager*Period',
       'Engineer*Period', 'Technician*Period', 'TECH*Period*Manager',
       'TECH*Period*Engineer', 'TECH*Period*Technician', 'Intercept',
       'plant_3m Odessa', 'plant_3m Saint Paul', 'plant_Aerojet Jupiter',
       'plant_Aerojet Los Angeles', 'plant_Aerospace El Segundo',
       'plant_Aerovironment Boston', 'plant_Aerovironment Burlington',
       'plant_Aerovironment Simi Valley', 'plant_Alcoa Austin'],
      dtype='object')


## Statistics

In [5]:
N = []
N.append(['Postings',f'{X.shape[0]:,}'])
N = pandas.DataFrame(N,columns=['Labels','N'])
N['Engineering Skills'] = numpy.nan
N.set_index(['Labels','N'],inplace=True)
N.columns = pandas.MultiIndex.from_product([N.columns,['A']])
statistics = []
for m in measures:
  statistics.append([m,f'{df[m].mean():.2f}',f'{df[m].std(ddof=1):.2f}'])
statistics = pandas.DataFrame(statistics,columns=['Measures','Mean','SD'])
statistics.set_index(['Measures'],inplace=True)
statistics = statistics.T
statistics.columns = pandas.MultiIndex.from_product([statistics.columns,['A']])
statistics.index = pandas.MultiIndex.from_product([statistics.index,['']])

## Poisson Regression

In [6]:
models = ['A','B','C']
myX = [XA,XB,XC]
mylist = [listA,listB,listC]
D = []
R = []
for model,x,l in zip(models,myX,mylist):
  for m in measures:
    reg0 = sm.Poisson(df[m],x).fit(method='cg',maxiter=1000)
    print(reg0.summary())
    reg = sm.Poisson(df[m],x).fit(method='cg',maxiter=1000,skip_hessian=True,cov_type='cluster',cov_kwds={'groups':Z})
    print(reg.summary())
    diff = 100*(reg.bse[:len(reg.bse)-len(set(Z))+1]-reg0.bse[:len(reg.bse)-len(set(Z))+1])/reg0.bse[:len(reg.bse)-len(set(Z))+1]
    diff = diff.mean()   
    test = overdispersion(x,m)
    R.append((model,m,reg.params[l],reg.bse[l],reg.pvalues[l],reg.prsquared,test[0],test[1],diff))
    D.append((model,m,reg.llf,reg.df_model))

Optimization terminated successfully.
         Current function value: 1.190719
         Iterations: 955
         Function evaluations: 1850
         Gradient evaluations: 1850
                          Poisson Regression Results                          
Dep. Variable:     Engineering Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212498
Method:                           MLE   Df Model:                          323
Date:                Thu, 13 May 2021   Pseudo R-squ.:                  0.1686
Time:                        12:02:16   Log-Likelihood:            -2.5341e+05
converged:                       True   LL-Null:                   -3.0480e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

Optimization terminated successfully.
         Current function value: 1.190719
         Iterations: 955
         Function evaluations: 1850
         Gradient evaluations: 1850
                          Poisson Regression Results                          
Dep. Variable:     Engineering Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212498
Method:                           MLE   Df Model:                          323
Date:                Thu, 13 May 2021   Pseudo R-squ.:                  0.1686
Time:                        12:09:06   Log-Likelihood:            -2.5341e+05
converged:                       True   LL-Null:                   -3.0480e+05
Covariance Type:              cluster   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

0.39817897710990147
Optimization terminated successfully.
         Current function value: 0.772705
         Iterations: 474
         Function evaluations: 1252
         Gradient evaluations: 1252
                          Poisson Regression Results                          
Dep. Variable:      Operations Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212498
Method:                           MLE   Df Model:                          323
Date:                Thu, 13 May 2021   Pseudo R-squ.:                  0.1936
Time:                        12:13:38   Log-Likelihood:            -1.6445e+05
converged:                       True   LL-Null:                   -2.0394e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

Optimization terminated successfully.
         Current function value: 0.772705
         Iterations: 474
         Function evaluations: 1252
         Gradient evaluations: 1252
                          Poisson Regression Results                          
Dep. Variable:      Operations Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212498
Method:                           MLE   Df Model:                          323
Date:                Thu, 13 May 2021   Pseudo R-squ.:                  0.1936
Time:                        12:18:19   Log-Likelihood:            -1.6445e+05
converged:                       True   LL-Null:                   -2.0394e+05
Covariance Type:              cluster   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

0.768869902128959
Optimization terminated successfully.
         Current function value: 2.437077
         Iterations: 622
         Function evaluations: 1105
         Gradient evaluations: 1105
                          Poisson Regression Results                          
Dep. Variable:         Support Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212498
Method:                           MLE   Df Model:                          323
Date:                Thu, 13 May 2021   Pseudo R-squ.:                 0.05109
Time:                        12:23:02   Log-Likelihood:            -5.1866e+05
converged:                       True   LL-Null:                   -5.4659e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------

...

0.09498349020654699
Optimization terminated successfully.
         Current function value: 1.766811
         Iterations: 701
         Function evaluations: 1216
         Gradient evaluations: 1216
                          Poisson Regression Results                          
Dep. Variable:         General Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212488
Method:                           MLE   Df Model:                          333
Date:                Thu, 13 May 2021   Pseudo R-squ.:                 0.07533
Time:                        15:01:30   Log-Likelihood:            -3.7602e+05
converged:                       True   LL-Null:                   -4.0665e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------

Optimization terminated successfully.
         Current function value: 1.766811
         Iterations: 701
         Function evaluations: 1216
         Gradient evaluations: 1216
                          Poisson Regression Results                          
Dep. Variable:         General Skills   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212488
Method:                           MLE   Df Model:                          333
Date:                Thu, 13 May 2021   Pseudo R-squ.:                 0.07533
Time:                        15:06:05   Log-Likelihood:            -3.7602e+05
converged:                       True   LL-Null:                   -4.0665e+05
Covariance Type:              cluster   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

0.037697648972125006
Optimization terminated successfully.
         Current function value: 1.713339
         Iterations: 837
         Function evaluations: 1456
         Gradient evaluations: 1456
                          Poisson Regression Results                          
Dep. Variable:         Job Complexity   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212488
Method:                           MLE   Df Model:                          333
Date:                Thu, 13 May 2021   Pseudo R-squ.:                 0.08092
Time:                        15:11:23   Log-Likelihood:            -3.6464e+05
converged:                       True   LL-Null:                   -3.9674e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------

Optimization terminated successfully.
         Current function value: 1.713339
         Iterations: 837
         Function evaluations: 1456
         Gradient evaluations: 1456
                          Poisson Regression Results                          
Dep. Variable:         Job Complexity   No. Observations:               212822
Model:                        Poisson   Df Residuals:                   212488
Method:                           MLE   Df Model:                          333
Date:                Thu, 13 May 2021   Pseudo R-squ.:                 0.08092
Time:                        15:16:53   Log-Likelihood:            -3.6464e+05
converged:                       True   LL-Null:                   -3.9674e+05
Covariance Type:              cluster   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------

-0.008706191085216112


## Model comparison

In [7]:
LLR = pandas.DataFrame(D,columns=['Model','Measure','ll','df']).set_index(['Model','Measure']).unstack()
LLR = LLR.diff(1)
for m in measures:
  LLR[m] = 1 - chi2.cdf(2*LLR.loc[:,('ll',m)],LLR.loc[:,('df',m)])
B = LLR.loc[['B'],[('Engineering Skills',''),('Operations Skills',''),('Support Skills',''),('General Skills',''),('Job Complexity','')]]
B = B.droplevel(1,axis=1)
B.columns = pandas.MultiIndex.from_product([['B'],B.columns])
B = B.reset_index(drop=True)
B['Variables'] = 'Model Comparison'
B['Statistics'] = ''
B.set_index(['Variables','Statistics'],inplace=True)
B = B.applymap('{:.2f}'.format) 
C = LLR.loc[['C'],[('Engineering Skills',''),('Operations Skills',''),('Support Skills',''),('General Skills',''),('Job Complexity','')]]
C = C.droplevel(1,axis=1)
C.columns = pandas.MultiIndex.from_product([['C'],C.columns])
C = C.reset_index(drop=True)
C['Variables'] = 'Model Comparison'
C['Statistics'] = ''
C.set_index(['Variables','Statistics'],inplace=True)
C = C.applymap('{:.2f}'.format)

## Table

In [8]:
tableA = Table_Summary(R[0:5])
tableA = tableA.reindex(listA+['Pseudo R-Squared','Overdispersion','Overdispersion p-value','Overdispersion correction'],level=0)
tableA = tableA.reindex(['Coeff','CSE','p-val',''],level=1)

tableB = Table_Summary(R[5:10])
tableB = tableB.reindex(listB+['Pseudo R-Squared','Overdispersion','Overdispersion p-value','Overdispersion correction'],level=0)
tableB = tableB.reindex(['Coeff','CSE','p-val',''],level=1)
tableB = tableB.append(B)

tableC = Table_Summary(R[10:15])
tableC = tableC.reindex(listC+['Pseudo R-Squared','Overdispersion','Overdispersion p-value','Overdispersion correction'],level=0)
tableC = tableC.reindex(['Coeff','CSE','p-val',''],level=1)
tableC = tableC.append(C)

tableB = pandas.merge(tableA,tableB,on=['Variables','Statistics'],how='outer')
table = pandas.merge(tableC,tableB,on=['Variables','Statistics'],how ='outer')
table = table.swaplevel(0,1,axis=1)
table = table.reindex(measures,axis=1,level=0).reindex(['A','B','C'],axis=1,level=1)

table = pandas.concat([table,statistics,N],axis=0)
table = table[measures]
table

Engineering Skills                  \
                                                      A       B       C   
Variables                 Statistics                                      
TECH                      Coeff                    0.60    0.83    0.96   
                          CSE                    (0.04)  (0.15)  (0.16)   
                          p-val                    0.00    0.00    0.00   
Manager                   Coeff                    0.07    0.04    0.04   
                          CSE                    (0.06)  (0.07)  (0.07)   
                          p-val                    0.24    0.56    0.59   
Engineer                  Coeff                    1.23    1.19    1.19   
                          CSE                    (0.06)  (0.07)  (0.08)   
                          p-val                    0.00    0.00    0.00   
Technician                Coeff                    0.71    0.85    0.86   
                          CSE                    (0.07)  (0.09)  (0.10)   
                          p-val                    0.00    0.00    0.00   
Period                    Coeff                   -0.01   -0.07   -0.06   
                          CSE                    (0.02)  (0.05)  (0.05)   
                          p-val                    0.49    0.21    0.25   
TECH*Manager              Coeff                     NaN    0.12    0.35   
                          CSE                       NaN  (0.17)  (0.22)   
                          p-val                     NaN    0.47    0.12   
TECH*Engineer             Coeff                     NaN   -0.16   -0.32   
                          CSE                       NaN  (0.15)  (0.18)   
                          p-val                     NaN    0.29    0.07   
TECH*Technician           Coeff                     NaN   -0.47   -0.83   
                          CSE                       NaN  (0.18)  (0.20)   
                          p-val                     NaN    0.01    0.00   
TECH*Period               Coeff                     NaN   -0.11   -0.27   
                          CSE                       NaN  (0.06)  (0.23)   
                          p-val                     NaN    0.07    0.25   
Manager*Period            Coeff                     NaN    0.04    0.05   
                          CSE                       NaN  (0.06)  (0.06)   
                          p-val                     NaN    0.45    0.41   
Engineer*Period           Coeff                     NaN    0.09    0.08   
                          CSE                       NaN  (0.05)  (0.06)   
                          p-val                     NaN    0.12    0.16   
Technician*Period         Coeff                     NaN   -0.20   -0.22   
                          CSE                       NaN  (0.07)  (0.08)   
                          p-val                     NaN    0.01    0.01   
TECH*Period*Manager       Coeff                     NaN     NaN   -0.32   
                          CSE                       NaN     NaN  (0.29)   
                          p-val                     NaN     NaN    0.28   
TECH*Period*Engineer      Coeff                     NaN     NaN    0.21   
                          CSE                       NaN     NaN  (0.24)   
                          p-val                     NaN     NaN    0.39   
TECH*Period*Technician    Coeff                     NaN     NaN    0.50   
                          CSE                       NaN     NaN  (0.30)   
                          p-val                     NaN     NaN    0.09   
Intercept                 Coeff                   -1.52   -1.49   -1.50   
                          CSE                    (0.05)  (0.06)  (0.06)   
                          p-val                    0.00    0.00    0.00   
Pseudo R-Squared                                 0.1686  0.1692  0.1693   
Overdispersion                                     0.40    0.40    0.40   
Overdispersion p-value                             0.00    0.0

In [9]:
table.to_excel('Table6.xlsx')